In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import pandas as pd

## all the functions

In [2]:
# switch the handle window from main page to question page
# click the question url to dive into question
def Into_QA(handle, page_index):
    '''
    switch the handle window from main page to question page
    click the question url to dive into question
    page_index: index of page desire to look into
    '''
    if handle.current_window_handle != parent_handle:
        handle.switch_to.window(handle.window_handles[0])
    
    page_url = "//*[@id='mainContent']/div[2]/div[2]/div[" +str(page_index)+"]"+"/div/div/div/div/div/div/div/div/div/div[1]/div/span/span/a/div/div/div/div"
    element = handle.find_element(By.XPATH, page_url )
    element.click()
    time.sleep(3)  
    # window_handles[1] is a second window
    handle.switch_to.window(handle.window_handles[1])

# 将handle从子页面切换回主页面,删掉handler释放内存
def out_QA(handle):
    '''
    close question page and move to parent page
    '''
    handle.close()
    handle.switch_to.window(parent_handle)




In [3]:
def transText(eList):
    '''
    change element list get from selenium into text list
    all the element should include the answer paragraph
    '''
    tmp = []
    for i in eList:
        tmp.append(i.text)
    return tmp

In [29]:
PAUSE_TIME =1.2
def data_acquire():
    '''
    acquire data from answer page.
    question: the string of question
    authorId : the webelement stores authorID
    upvote : the webelement stores upvote
    answer : the webelement stores answer
    answerb : for different html versions, 
                some answers are stored in different structure of Quora html. 
                this would retrieve answer data if the answer didn't work. 
    '''
    time.sleep(PAUSE_TIME)
    question = wd.find_element(By.XPATH,"//*[@id='mainContent']/div/div[1]/div/div/div/div[1]/span/span/div/div/div/span/span").text
    authorID = wd.find_elements(By.XPATH,'//*[@id="mainContent"]/div/div[2]/div[*]/div/div/div/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div[2]/div[1]/span/span')
    answer = wd.find_elements(By.XPATH,'//*[@id="mainContent"]/div/div[2]/div[*]/div/div/div/div/div/div/div/div/div[1]/div[2]/div/div[1]')
    upvote = wd.find_elements(By.XPATH,'//*[@id="mainContent"]/div/div[2]/div[*]/div/div/div/div/div/div/div/div/div[2]/div/div/div[1]/div/div[1]/div/button/div[2]/span[2]')
    answerb = wd.find_elements(By.XPATH,'//*[@id="mainContent"]/div/div[2]/div[*]/div/div/div/div/div/div/div/div/div[1]/div[3]/div/div[1]')
    return question,authorID,answer,upvote,answerb
    

In [42]:
# changing a piece of data into dataframe
def to_dataframe(question, authorID, answer, vpvote,answerb):
    '''
    changing data into dataframe type
    deal with cama and other data type problems 
    '''
    authorID = transText(authorID)
    answer = transText(answer)
    if len(authorID) != len(answer) and len(answerb)!=0:
        answer.extend(transText(answerb)) 
        
    vpvote = transText(vpvote)
    vote = ['0']* len(answer) # make upvote default zero
    vote[:len(vpvote)] = vpvote

    df =pd.concat([pd.DataFrame({'Question':[question]}), pd.DataFrame({'authorID':authorID}), pd.DataFrame({'content':answer}),pd.DataFrame({'upvote':vote})],axis=1)
    
    df['Question'] =  df['Question'].fillna(value=question)
    df['rank'] = range(1,len(df)+1)
    df[['Question','content']] = df[['Question','content']].apply(lambda x: x.str.replace(',',' '))
    df['content'] = df['content'].apply(lambda x: x.replace('\n',''))  
    return df
    

In [58]:
def writeData(dataframe,index):
    '''
    record the data into local drive, chaning the location before running progress.
    
    '''
#     location = 'd:\\Quora_scraper\\softskills and Autism\\'+str(index)+'.txt'
#     location = 'd:\\Quora_scraper\\Cryptocurrency\\'+str(index)+'.txt'
    location = 'd:\\Quora_scraper\\softwareengineering\\'+str(index)+'.txt'
    dataframe.to_csv(location, index=None, sep=',', mode='a')

In [7]:
# sort question dispaly in recent order
def switch_sorting(page_index):
    '''
    select the recent sort method
    don't need page_index only when to report bug
    '''

    try:
        sorts = wd.find_elements(By.XPATH,"//*[@id='mainContent']/div/div[2]/div[1]/div/div[2]/div/div[2]/div/div/button/div")
        if len(sorts)==0:
            sort = wd.find_element(By.XPATH,"//*[@id='mainContent']/div/div[2]/div[1]/div/div[2]/div/div/div/button/div/div[2]/div")
            sort.click()
            sort_recent = wd.find_element(By.XPATH,"//*[@id='mainContent']/div/div[2]/div[1]/div/div[2]/div/div/div[2]/div/div[1]/div/div/div[3]/div/div")
            sort_recent.click()
        else:
            sorts[0].click()
            sort_recent = wd.find_element(By.XPATH,"//*[@id='mainContent']/div/div[2]/div[1]/div/div[2]/div/div[2]/div/div[2]/div/div[1]/div/div/div[3]/div[1]")
            sort_recent.click()
    except:
        print('no answer',page_index)
        pass
        

In [8]:
def hasAnswered(page_index):
    '''
    Determine if the question has an answer
    '''
    if wd.current_window_handle != parent_handle:
        wd.switch_to.window(wd.window_handles[0])
    
    AnswerIndex = "//*[@id='mainContent']/div[2]/div[2]/div[" +str(page_index)+"]"+"/div/div/div/div/div/div/div/div/div/div[2]/span[1]/a"
    elements = wd.find_elements(By.XPATH, AnswerIndex )
    if elements[0].text != 'No answer yet':
        return True
    else:
        return False


In [48]:
def ScrolltoCertain(page_index):
    '''
    to avoid start from zero, scroll down to desire part
    page_index: which answer you want start, like if I want to start from 50th question, page_index=50,
                the progress would scroll down until the 50th question shows on screen.
    '''
    if wd.current_window_handle != parent_handle:
        wd.switch_to.window(wd.window_handles[0])
    findFlag = True
    while (findFlag):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(PAUSE_TIME)
        page_url = "//*[@id='mainContent']/div[2]/div[2]/div[" +str(page_index)+"]"+"/div/div/div/div/div/div/div/div/div/div[1]/div/span/span/a/div/div/div/div"
        targetPage = wd.find_elements(By.XPATH, page_url )
        if len(targetPage) !=0:
            findFlag =False

# processing

In [59]:
# inititalize the drive
# create WebDriver objects，and define Chrome as browser
wd = webdriver.Chrome(service=Service(r'D:\chromedriver_win32\chromedriver.exe'))

# Select three Quora space to get all the answers and questions

# wd.get('https://softskillsandautism.quora.com/questions')
# wd.get('https://cryptocurrency.quora.com/questions')
wd.get('https://softwareengineeringexperiences.quora.com/questions')

wd.implicitly_wait(3)
parent_handle = wd.current_window_handle

page_index = 1
# if certain webpage doesn't work, could set certain page in stop_page to move forward
# stop_page = [79,122]
stop_page = []

# if you want to start from certain page, use this function to scroll down to desire page
# ScrolltoCertain(page_index)

flag = True
while (flag):
    if hasAnswered(page_index) == False or page_index in stop_page:
        page_index +=1
        print('leave one unanswered question ',page_index)
        continue
    else:
        try:

            Into_QA(wd, page_index)
        except:
            print("Error in entering into %d",page_index)
            flag = False

        switch_sorting(page_index)
        # if there is hidden content, show all 
        fullContent = wd.find_elements(By.XPATH,'//*[@id="mainContent"]/div/div[2]/div[*]/div/div/div/div/div/div/div/div/div[1]/div[*]/div/div/div[2]/div[1]/div/div/div/span/div/div/div/button/div/div/div')
        for i in fullContent:
            i.click()

        ques,author,answer,upvote,answerb = data_acquire()  # retrieve data
        result = to_dataframe(ques,author,answer,upvote,answerb) #make a dataframe
        print(result)
        writeData(result,page_index) #store
        out_QA(wd)

        page_index = page_index+1

leave one unanswered question  2
leave one unanswered question  3
                                            Question     authorID  \
0  As a software engineer  how do you measure bur...  Alex Eustis   

                                             content upvote  rank  
0  This is a pretty good summary here:Know the si...      0     1  
leave one unanswered question  5
                                            Question           authorID  \
0  Can people with a fixed mindset succeed as a s...  Demont Washington   

                                             content upvote  rank  
0  tl dr: NoTo be an engineer in any realm is to ...      0     1  
                                            Question            authorID  \
0  Why is the average tenure of a software engine...  Francesco Fittizio   

                                             content upvote  rank  
0  I think it's because once you got one of the l...      0     1  
                                            Questi

                                            Question          authorID  \
0  As a software engineer  do you try to upskill ...  Victor Mikhailov   

                                             content upvote  rank  
0  I never had downtime more than 1 month.But dur...      0     1  
                                            Question        authorID  \
0  When Ian Clark says  "SQL was created for a re...  Mariano Curros   

                                             content upvote  rank  
0  Yes  SQL was created for a reason:As common la...      0     1  
                                            Question            authorID  \
0  Does pair programming work when it is enforced...   Demont Washington   
1  Does pair programming work when it is enforced...          Matt Erman   
2  Does pair programming work when it is enforced...    Jake Van Wagoner   
3  Does pair programming work when it is enforced...       Joseph Rogers   
4  Does pair programming work when it is enforced...  

                                            Question            authorID  \
0  Should software engineers focus on building th...  Michael McAllister   
1  Should software engineers focus on building th...               AlanM   
2  Should software engineers focus on building th...   Demont Washington   
3  Should software engineers focus on building th...          Chuck Cobb   

                                             content upvote  rank  
0  I don't know if it's worth focusing on  but it...      3     1  
1  Before you spend time trying to create a perso...      3     2  
2  Every software professional should have a bran...      1     3  
3  There’s probably not a lot of value in that un...      0     4  
                                            Question authorID  \
0  As a software engineer  should one always rais...   Nick M   
1  As a software engineer  should one always rais...  S Ahmed   

                                             content upvote  rank  
0  Yes and no.I

                                            Question         authorID  \
0  I’m a junior software developer that started w...        Otto Gold   
1  I’m a junior software developer that started w...  Andrea Raimondi   
2  I’m a junior software developer that started w...     Lynn Wallace   

                                             content upvote  rank  
0  There are three ways how to do this. The typic...      0     1  
1  HA!I know I may be perceived as defeatist but…...      0     2  
2  Walk them through your code. Start with the st...      0     3  
leave one unanswered question  72
                                            Question      authorID  \
0  How can a software engineering tech lead avoid...        Nick M   
1  How can a software engineering tech lead avoid...  Karl Jansson   
2  How can a software engineering tech lead avoid...    Greg Young   

                                             content upvote  rank  
0  Never be the smartest person in the room.If yo...

                                            Question         authorID  \
0  As a software engineering interviewer  do you ...  William Richter   

                                             content upvote  rank  
0  I’ve run into this situation. Too many questio...      0     1  
leave one unanswered question  101
leave one unanswered question  102
                                            Question        authorID  \
0  Should a software engineer refrain from compar...  Steven Knudsen   

                                             content upvote  rank  
0  If the answer to x > 4 is unknown on your syst...      0     1  
leave one unanswered question  104
leave one unanswered question  105
                                            Question            authorID  \
0  What's the difference between a Full Stack Eng...  Wallace B. McClure   
1  What's the difference between a Full Stack Eng...      Steven J Owens   

                                             content upvote  rank  

...

leave one unanswered question  914
                                            Question     authorID  \
0  Would a software engineer do everything to sol...  David Sharp   
1  Would a software engineer do everything to sol...  Bryan White   

                                             content upvote  rank  
0  A professional software engineer would realize...      5     1  
1  Yes. I have faced this situation on occasions....      0     2  
leave one unanswered question  916
                                            Question           authorID  \
0  As a software engineer  how do you go about de...       James Barton   
1  As a software engineer  how do you go about de...          Rob James   
2  As a software engineer  how do you go about de...  Michael Lloyd Lee   

                                             content upvote  rank  
0  That's rather simple actually.ALL programmers ...      2     1  
1  That can be a problem. Diagnostomorphosis is a...      0     2  
2  Does the e

                                            Question authorID  \
0  How can a software engineer develop a testing ...    KaliO   

                                             content upvote  rank  
0  Well if the developers are held accountable fo...      0     1  
                                            Question            authorID  \
0  When was the Y2K problem first identified by s...  Wallace B. McClure   
1  When was the Y2K problem first identified by s...         Bryan White   
2  When was the Y2K problem first identified by s...       John Mezzetta   
3  When was the Y2K problem first identified by s...           Dan Koren   

                                             content upvote  rank  
0  The problem was known about for as long as dat...     12     1  
1  Before 1974. That's when I was told about it  ...      4     2  
2  The limitation was known from day one.Along th...      1     3  
3  Apparently some time around 1993:Year 2000 pro...      0     4  
leave one un

                                            Question             authorID  \
0  Are majority of candidates who apply for softw...  Håkon Hapnes Strand   
1  Are majority of candidates who apply for softw...            Rob James   

                                             content upvote  rank  
0  Honestly  I don't expect my candidates to be p...      1     1  
1                                             Kinda.      0     2  
                                            Question            authorID  \
0  As a software engineer when your fellow engine...  Francesco Fittizio   

                                             content upvote  rank  
0  Yes. If there are comments  I won't approve un...      0     1  
                                            Question        authorID  \
0  Are software engineers most productive on no m...  James H. Kelly   

                                             content upvote  rank  
0  It depends on how you define productive. If pr...      0   

                                            Question authorID  \
0  How can I add value to an already solved problem?   Ilam P   

                                             content upvote  rank  
0  One of the best questions I have seen this wee...      0     1  
                                            Question          authorID  \
0  Have you ever made a disassembler for a progra...  Bruce F. Webster   

                                             content upvote  rank  
0  Asked and answered:Software Engineering Experi...      0     1  
leave one unanswered question  1028
                                            Question         authorID  \
0  As a software engineering interviewer  does ho...  Kurt Guntheroth   

                                             content upvote  rank  
0  Wow  that’s a hard one. I like candidates who ...      0     1  
                       Question           authorID  \
0  Is scrum better than kanban?              KaliO   
1  Is scrum better th

IndexError: list index out of range